In [9]:
## This python file only contains one example of transforming and projecting the candidate profile to the recruiting matrix.

In [10]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler

import json
import pandas as pd

In [11]:
# import cleaned candidate dataset:
with open("candidate_profile.json","r") as f:
    candidate_data = json.load(f)

candidate_data

{'10000': {'Name': 'lindsey weedon',
  'Country&City': 44,
  'Work_Autho': 'None',
  'Degree&Major': [[14, ['business management', 'management', 'business']]],
  'YOE&Industry': [[123, ['marketing manager']],
   [365, ['manager']],
   [396, ['manager']],
   [303, ['marketing manager']],
   [395, ['marketing manager']],
   [853, ['founder']],
   [365, ['events manager']],
   [61, ['marketing manager']],
   [853, ['marketing specialist']],
   [123, ['manager']],
   [577, ['coordinator']]],
  'skills': [],
  'working_experiences': ['marketing contractor responsibilitie global adsense - small - medium publisher marketing lead global marketing strategy line adsense business product priority drive acquisition growth',
   'act subject matter expert adsense marketing collaborating multiple stakeholder write narrative provide strategic guidance channel owner website blog email social medium youtube seo pay',
   'work closely product sale go - to - market team align consult wide commercialisatio

In [12]:
# import hard skills dataset:
with open("/Users/zhiyuzhang/Desktop/ocbang/Matching System/O*NET web scraping/final_merge&analysis/hardskills.json","r") as f:
    hardskills = json.load(f)

hardskills

[{'label': 'File versioning software', 'pattern': 'zyimage'},
 {'label': 'File versioning software', 'pattern': 'version control software'},
 {'label': 'File versioning software', 'pattern': 'zylab zyimage'},
 {'label': 'File versioning software', 'pattern': 'winmerge'},
 {'label': 'File versioning software', 'pattern': 'git'},
 {'label': 'File versioning software', 'pattern': 'version control'},
 {'label': 'File versioning software',
  'pattern': 'concurrent versions systems'},
 {'label': 'File versioning software', 'pattern': 'continuous integration'},
 {'label': 'File versioning software', 'pattern': 'apache subversion'},
 {'label': 'File versioning software', 'pattern': 'subversion'},
 {'label': 'File versioning software',
  'pattern': 'continuous integration software'},
 {'label': 'Data mining software', 'pattern': 'golden helix chemtree'},
 {'label': 'Data mining software', 'pattern': 'ibm'},
 {'label': 'Data mining software', 'pattern': 'egrabber listgrabber'},
 {'label': 'Data 

In [13]:
# import soft skills dataset:
with open("softskills.json","r") as f:
    softskills = json.load(f)

softskills

{'parser': ['processing information'],
 'optical scanner': ['processing information'],
 'coder': ['processing information'],
 'publish': ['processing information',
  'working with computers',
  'communicating with people outside the organization',
  'communicating with supervisors, peers, or subordinates',
  'documenting/recording information'],
 'verifie': ['processing information'],
 'verify': ['processing information'],
 'external auditor': ['processing information'],
 'tally': ['processing information'],
 'substantiate': ['processing information'],
 'bookkeeping': ['processing information'],
 'distinguish': ['processing information',
  'training and teaching others',
  'analyzing data or information',
  'coaching and developing others',
  'inspecting equipment, structures, or materials',
  'identifying objects, actions, and events'],
 'format': ['processing information'],
 'verification': ['processing information',
  'drafting, laying out, and specifying technical devices, parts, a

In [14]:
nlp = English()
ruler = EntityRuler(nlp, validate=True)
# ruler.add_patterns(patterns)
entity_ruler = nlp.add_pipe("entity_ruler")
entity_ruler.initialize(lambda: [], nlp=nlp, patterns=hardskills)

In [15]:
def most_frequent(List):
    counter = 0
    num = List[0]
     
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
 
    return num

### show 00001 as an example

In [17]:
candidate_skills_dictionary = {}
# only deal with work experience
for id in candidate_data.keys():
    softskills_temp = []
    hardskills_temp = []
    # list of work experience
    work_experience = candidate_data[id]['working_experiences']
    for each_sentence in work_experience:
        # hard skill recognition
        doc = nlp(each_sentence)
        for ent in doc.ents:
            hardskills_temp.append(ent.label_)
        # soft skill recognition
        temp_list_inner = []
        for each_word in each_sentence.split():
            if each_word in softskills.keys():
                skill = softskills[each_word]
                temp_list_inner = temp_list_inner+skill
            else:
                continue
        if temp_list_inner == []:
            softskills_temp = softskills_temp
        elif len(temp_list_inner) == len(list(set(temp_list_inner))):
            softskills_temp = softskills_temp
        else:
            softskills_temp.append(most_frequent(temp_list_inner))
    print(list(set(hardskills_temp)))
    print(list(set(softskills_temp)))
    break

['Business intelligence and data analysis software', 'Project management software', 'Network conferencing software', 'Compliance software', 'Object or component oriented development software', 'Data base management system software', 'Desktop publishing software', 'Data base reporting software', 'Accounting software', 'Enterprise resource planning ERP software', 'Document management software', 'Graphics or photo imaging software', 'Video creation and editing software', 'Data base user interface and query software', 'Computer based training software', 'Expert system software', 'Development environment software', 'Electronic mail software', 'WAN switching software and firmware', 'Customer relationship management CRM software', 'Analytical or scientific software', 'Cloud-based data access and sharing software', 'Calendar and scheduling software']
['estimating the quantifiable characteristics of products, events, or information', 'coaching and developing others', 'handling and moving object

In [19]:
### show candidate "000001"
work_experience = candidate_data[id]['working_experiences']
work_experience

['marketing contractor responsibilitie global adsense - small - medium publisher marketing lead global marketing strategy line adsense business product priority drive acquisition growth',
 'act subject matter expert adsense marketing collaborating multiple stakeholder write narrative provide strategic guidance channel owner website blog email social medium youtube seo pay',
 'work closely product sale go - to - market team align consult wide commercialisation plan',
 'lead research drive content development improve pay organic acquisition campaign',
 'increase acquisition conversion - month pay medium beta campaign update creative improving target',
 'manage marketing budget - instal process rigor improve roi reporting spend efficiency',
 'central approver adsense global marcomms ensure google brand continuity consistency throughout',
 'lead think leadership initiative sale stakeholder drive centralised cross - channel approach - focus proactive engage helpful simplified publisher comm

In [21]:
# visualization hard skills
from spacy import displacy
for i in work_experience:
    doc = nlp(i)
    # options = {'colors':{"Operating system software":"#f2aef1","Development environment software":"#0cf7a9","Object or component oriented development software":"#a5c621","Data base management system software":"#477899"}}
    displacy.render(doc,style="ent",jupyter=True)

# Important Problem! (some common words are mistakenly recognized as hardskills like go, c, publisher, act, drive, reporting, partener, legal.)
# one solution of this: firstly: distinguish the word properties (do not recognize VERB) and do recognition/ refine the hardskills dataset

/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


...

/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [ ]:
# This part of code can be used to transform all candidates profile
"""
candidate_skills_dictionary = {}
# only deal with work experience
for id in candidate_data.keys():
    softskills_temp = []
    hardskills_temp = []
    # list of work experience
    work_experience = candidate_data[id]['working_experiences']
    for each_sentence in work_experience:
        # hard skill recognition
        doc = nlp(each_sentence)
        for ent in doc.ents:
            hardskills_temp.append(ent.label_)
        # soft skill recognition
        temp_list_inner = []
        for each_word in each_sentence.split():
            if each_word in softskills.keys():
                skill = softskills[each_word]
                temp_list_inner = temp_list_inner+skill
            else:
                continue
        if temp_list_inner == []:
            softskills_temp = softskills_temp
        elif len(temp_list_inner) == len(list(set(temp_list_inner))):
            softskills_temp = softskills_temp
        else:
            softskills_temp.append(most_frequent(temp_list_inner))
    candidate_skills_dictionary[id] = {"hardskills":hardskills_temp,"softskills":softskills_temp}
"""

### Things to be done next:
- Provide more JDs and improve the data cleaning of JD.

- How to deal with other features other than soft and hard skills. (case by case: importance/ how to deal)

- Continually improve four datasets (hardskills/softskills/job title/major) and corresponding problems listed above.

- Provide combination of JD AND selected & unqualified candidates to train the models (find best hyperparameters with accuracy score)!